# EXTRACCIÓN DE LOS DATOS DE YOUTUBE

Para la red social de Youtube, se extraerán datos con un único  script.<br>
- Script para sacar información de los perfiles y videos de Youtube introducirlos en la BBDD

## PSEUDOCÓDIGO

**El psudocódigo del programa será de la siguiente manera**

Abrir Base de Datos
Seleccionar y extraer los datos de la tabla donde se almacenan los nombres de las diferentes cuentas de Youtube<br><br>
Extraer las cuentas de youtube<br><br>
Para cada una de las cuentas:<br>
   &emsp; encontrar los enlaces de los videos<br>
   &emsp; para cada uno de los enlaces de los videos:<br>
   &emsp; &emsp; extraer los datos de la cuentas<br>
   &emsp; &emsp; &emsp; titulo, nombre, duracion, visualizaciones, fecha
   &emsp; &emsp; introducir los datos en la tabla correspondiente<br><br>

## LIBRERÍAS

Para la correcta extracción y recopilación de los datos, se aplicarán diferentes librerías pero la librería principal es:<br><br>
**__PYTUBE__**: La librería de Pytbe es una librería especializada en descargar y obtener información de los videos de Youtube, se puede consultar la documentación  __[aquí](https://pytube.io/en/latest/)__.<br><br>
**__SELENIUM__**: Es un entorno de pruebas de software para aplicaciones basadas en la web. Su función principal es realizar web scraping de manera automatizada mediante diferentes métodos posibles como HTML, CSS, XPATH, etc..<br><br>Esto es gracias a la implantación de un driver que automatizará el navegador y que se puede descargar __[aquí](https://selenium-python.readthedocs.io/installation.html)__.<br><br>
La documentación de Selenium para Python se puede consultar __[aquí](https://selenium-python.readthedocs.io/getting-started.htmly)__.<br>

La librería selenium no es como otras librerías usadas en otras redes sociales que son especializadas en la extracción de datos en esa red social, selenium recorre la web y es el programador el que recoge los datos.<br><br>

Otras librerías que se usan son:
- **Pandas** : Manipulación de datos
- **Mysql.connector**: Conexión con la base de datos
- **Codecs**: Normalización de caracteres


## PASOS PREVIOS

Para realizar estos scripts, primeramente se añade una tabla a la BBDD con la siguiente sintaxis:

- **TABLA PARA VIDEOS YOUTUBE**:

CREATE TABLE IF NOT EXISTS videos_yotube(<br>
    &emsp; ID INT AUTOINCREMENT,<br>
    &emsp; Universidad VARCHAR(100),<br>
	&emsp; Nombre VARCHAR(50),<br>
    &emsp; Titulo VARCHAR(200),<br>
    &emsp; Descripcion VARCHAR(200),<br>
    &emsp; Fecha DATETIME,<br>
    &emsp; Seguidores BIGINT,<br>
    &emsp; Visualizaciones BIGINT,<br>
    &emsp; Duracion BIGINT,<br>
    &emsp; Palabras_Clave TEXT,<br>
    &emsp; PRIMARY KEY(ID))<br>
)ENGINE = INNODB;<br>



## CÓDIGO

Ya que se tiene todo los requisitos, ahora se abre la conexión con la Base de Datos y se empieza a codificar. <br>
Lo primero de todo es importar las librerias que necesitamos

In [ ]:
import pandas as pd
import mysql.connector
import codecs
from pytube import YouTube
codecs.register(lambda name: codecs.lookup('utf8') if name == 'utf8mb4' else None)

In [ ]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Acto seguido, se abre la conexión de la base de Datos con Python 

In [ ]:
database = mysql.connector.connect(
    host= "localhost",
    user= "root",
    passwd= "",
    database= "TFG",
    use_unicode = True, 
    charset="utf8mb4",
    collation = "utf8mb4_general_ci"
)
database.set_charset_collation("utf8mb4","utf8mb4_general_ci")

cursor= database.cursor()

Se extraen las cuentas de Youtube de la tabla de la Base de Datos **"universidades"**

In [ ]:
datos = pd.read_excel(r"C:\Users\cesar\Desktop\Scraping TFG\SNspaUNIV.xlsx", sheet_name= "WEB_SN")
cuentas_youtube = datos["Youtube"].tolist()
universidad = datos["Universidad"].tolist()
for a in range(len(cuentas_youtube)-1,-1,-1):
    if type(cuentas_youtube[a]) != str:
        cuentas_youtube.remove(cuentas_youtube[a])
        universidad.remove(universidad[a])
print( len(cuentas_youtube),len(universidad))

Como no todas las Universidades tienen youtube, se realiza una limpieza.<br>
Ahora se empieza a codificar empezando con selenium para recoger los enlaces de los videos.

In [ ]:
opts= Options()
opts.add_argument("--user-agent=Mozilla/5.0 (iPad; CPU OS 6_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/6.0 Mobile/10A5355d Safari/8536.25")
opts.add_argument("--start-maximized")
opts.add_experimental_option('excludeSwitches', ['enable-logging'])

driver= webdriver.Chrome('./chromedriver.exe', options=opts)
        

Se generan las opciones y se crea el driver que servirá para crear el navegador.
Acto seguido se implementa el código entero con un bucle anidado de nivel 2 en el que el primer bucle recoge los enlaces de los videos y el interno recoge la cada información de cada enlace.

In [ ]:
for a in range(len(cuentas_youtube)):
    driver.get(cuentas_youtube[a])## empieza escogiendo una universidad
    try: # lo primero de todo es logearse si no se está y la página emerge una solicitud de  autentificación
        driver.find_element_by_css_selector('a.WpHeLc.VfPpkd-mRLv6.VfPpkd-RLmnJb').click()
        usuario =  driver.find_element(By.XPATH, '//input[@name="identifier"]')
        usuario.send_keys("100383702@alumnos.uc3m.es")
        driver.find_element(By.XPATH, '//button[@name="action"]').click()
        password = driver.find_element(By.XPATH, '//input[@name="Passwd"]')
        password.send_keys("cesar999")
        driver.find_element(By.XPATH, "//button[@name = 'action']").click()
    except:
        try:
            driver.find_element(By.XPATH, "//button[@name = 'Email']").click()
            password = driver.find_element(By.XPATH, '//input[@name="Passwd"]')
            password.send_keys("cesar999")
            password = driver.find_element(By.XPATH, '//input[@class="MK9CEd MVpUfe"]').click()
        except:
            pass
    sleep(2)#recofemos el titulo y los seguidores del perfil
    titulo = driver.find_element_by_css_selector('h1.c4-tabbed-header-title').text
    seguidores = driver.find_element_by_css_selector('span.c4-tabbed-header-subscriber-count.secondary-text').text
    try:# se procesan y limpian
        seguidores = int(seguidores.split()[0])
    except:
        try:
            seguidores = seguidores.split()[0]
            seguidores = int(seguidores.replace(".",""))
        except:
            seguidores = None
    driver.find_elements_by_css_selector('a.scbrr-tab.center')[1].click()
    sleep(2)

    screen_height = driver.execute_script("return window.screen.height;")
    i = 1
    while True: #hacemos scroll hasta que se acabe la pagina
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        sleep(1)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            sleep(5)
            scroll_height = driver.execute_script("return document.body.scrollHeight;")
            if (screen_height) * i > scroll_height:
                break
    sleep(1)

    videos = driver.find_elements_by_xpath("//a[@class='compact-media-item-metadata-content']") #se recogen los contenedores de los videos
    links = []
    for video in videos:
        links.append(video.get_attribute("href"))# se recogen los enlaces
    print(len(links))

    for b in range(len(links)):#con la libreria de pytube se extaren los datos
        yt = YouTube(links[b])
        datos = [universidad[a],titulo,seguidores,yt.title,yt.description," ; ".join(yt.keywords), yt.length, yt.publish_date.strftime('%Y-%m-%d'),yt.views, yt.rating]
        
        #se introducen los datos
        cursor.execute("INSERT INTO videos_youtube VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",datos)
        database.commit()


### Con este pequeño script están los datos de la red social Youtube extraidos.<br><br> Duración aproximada 2h